In [46]:
import pandas 

data_salary = pandas.read_csv('salary-train.csv')

## Предобработка данных

Приведите тексты к нижнему регистру (text.lower()).

Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты:

In [47]:
data_salary["FullDescription"] = data_salary["FullDescription"].apply(lambda x: x.lower())
data_salary["FullDescription"] = data_salary["FullDescription"].replace('[^a-zA-Z0-9]', ' ', regex = True)


Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfi = TfidfVectorizer(min_df=5)
scaled = tfi.fit_transform(data_salary["FullDescription"])

<60000x22861 sparse matrix of type '<type 'numpy.float64'>'
	with 8365759 stored elements in Compressed Sparse Row format>

Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'.

In [49]:
data_salary['LocationNormalized'].fillna('nan', inplace=True)
data_salary['ContractTime'].fillna('nan', inplace=True)

Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

In [50]:
from sklearn.feature_extraction import DictVectorizer

enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_salary[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_train_categ


<60000x1766 sparse matrix of type '<type 'numpy.float64'>'
	with 120000 stored elements in Compressed Sparse Row format>

Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными.

In [51]:
from scipy.sparse import hstack

matrix = hstack([scaled, X_train_categ])

Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

In [52]:
from sklearn.linear_model import Ridge
import numpy as np

clf = Ridge(alpha=1, random_state=241)
clf.fit(matrix, data_salary['SalaryNormalized'])

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

## Предобработка тестовых данных

In [72]:
data_test = pandas.read_csv('salary-test-mini.csv')

data_test["FullDescription"] = data_test["FullDescription"].apply(lambda x: x.lower())
data_test["FullDescription"] = data_test["FullDescription"].replace('[^a-zA-Z0-9]', ' ', regex = True)

scaled_test = tfi.transform(data_test["FullDescription"])

data_test['LocationNormalized'].fillna('nan', inplace=True)
data_test['ContractTime'].fillna('nan', inplace=True)

X_test = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

matrix_test = hstack([scaled_test, X_test])

Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание

In [74]:
res = clf.predict(matrix_test)
print round(res[0], 2)
print round(res[1], 2)

56555.62
37188.32
